<small>Este material foi produzido para o *Curso Básico de Google Earth Engine no Python (usando o Google Colab )*. https://www.github.com/andrebelem/Curso_Basico_Geemap. Versão 1.0 | © 2024 Andre Belem
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andrebelem/Curso_Basico_Geemap/blob/main/Modulo03_Animacoes_de_timelapse.ipynb). Se você tem algum tipo de dúvida ou quiser fazer um comentário, sugestão ou colaborar com este notebook... [![Mail Me](https://img.shields.io/badge/Envie_um-Email-blue)](mailto:andrebelem@id.uff.br)</small>

## Autenticando ...

**Dica**: Você já deve saber como fazer isso !

In [ ]:
import ee
import geemap
ee.Authenticate()

In [ ]:
# Aqui vocẽ deve trocar o projeto pelo "SEU-PROJECT-ID"
ee.Initialize(project='ee-andrebelem')

In [ ]:
# para as animações de timelapse, vamos precisar do ffmpeg.
# Note que isso é importante para o COLAB, mas vocẽ deve adaptar na sua máquina local caso precise
!pip install ffmpeg-python

## Precisamos de dados de uma coleção

Uma animação de timelapse no Geemap é uma técnica visual utilizada para representar mudanças ao longo do tempo em dados geoespaciais. Primeiro, você precisa de uma coleção de imagens que cubra o período desejado. Essas imagens podem ser de satélites, drones ou qualquer outra fonte que forneça dados geoespaciais ao longo do tempo, mas que estejam na forma de `assets` do `ee`.
Pense no que você quer fazer. Antes de criar a animação, pode ser necessário processar as imagens, o que inclui operações como normalização, filtragem de nuvens, cálculo de índices (por exemplo, NDVI para vegetação), entre outros.
O último passo é a *Configuração da Animação*. No `geemap`, você define os parâmetros da animação, como intervalo de tempo, região de interesse, parâmetros de visualização e a frequência dos frames.


Vamos começar definindo nossa área de interesse (ou região de interesse), e vou escolher novamente, a *Floresta da Tijuca* no Rio de Janeiro.

### Definindo uma região de interesse (ROI)

Aqui temos duas opções: você pode definir sua Região de Interesse (ROI) por código, se você já sabe o que quer, ou interativamente na imagem. Note que a sequência é 1. abrir o mapa, 2. definir um roi., 3. ativar o timelapse.

In [ ]:
map = geemap.Map(center=[-22.85, -43.16], zoom=10)
map.add_basemap("Esri.WorldImagery")
map

Navegue e amplie o mapa para uma área de interesse (no nosso caso, ao redor da Floresta da Tijuca, bem no meio da cidade do Rio de Janeiro). Use as ferramentas de desenho para desenhar um retângulo no mapa. Se nenhum retângulo for desenhado, o retângulo padrão mostrado abaixo será usado.

In [ ]:
# definindo o roi
roi = map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-43.35, -23.05, -43.19, -22.89)
    map.addLayer(roi)
    map.centerObject(roi)

Agora, vamos às imagens:

### Landsat

Essa animação usa imagens do Landsat de um conjunto de vários datasets que contém imagens de reflectância de superfície (Surface Reflectance, SR), entre eles Landsat-4, -5, -7 e -8. Veja mais detalhes [aqui](https://github.com/gee-community/geemap/blob/master/geemap/timelapse.py).

In [ ]:
# basta usar a função abaixo...
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif="Floresta_da_Tijuca.gif",
    start_year=2000,
    end_year=2023,
    start_date="06-01",
    end_date="09-01",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=5,
    title="Landsat Timelapse",
    progress_bar_color="blue",
    mp4=True,
)
geemap.show_image(timelapse)

### MODIS NDVI

Note que já existe uma função específica para o MODIS no geemap, que trabalha sobre o produto [MODIS Terra Vegetation Indices 16-Day Global 1km dataset](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A2). A resolução não é muito boa ! Então note que ela serve para áreas maiores.

In [ ]:
map = geemap.Map(center=[-22.85, -43.16], zoom=10)
map.add_basemap("Esri.WorldImagery")
map

In [ ]:
# definindo o roi
roi = map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-43.35, -23.05, -43.19, -22.89)
    map.addLayer(roi)
    map.centerObject(roi)

In [ ]:
timelapse = geemap.modis_ndvi_timelapse(
    roi,
    out_gif="ndvi.gif",
    data="Terra",
    band="NDVI",
    start_date="2000-01-01",
    end_date="2022-12-31",
    frames_per_second=3,
    title="MODIS NDVI Timelapse",
    overlay_data="countries",
)
geemap.show_image(timelapse)

## Explorando outras animações com o `geemap`

Se você olhar o manual do `geemap`, vai notar que há uma série de scripts prontos que vão desde dados do *GOES fire* (que é um sistema de detecção de queimadas), até uma série temporal do *Sentinel-2*. Vamos mostrar aqui mais um timelapse ...

In [ ]:
# iniciamos da mesma forma, centrando no Rio de Janeiro

map = geemap.Map(center=[-22.85, -43.16], zoom=10)
map.add_basemap("Esri.WorldImagery")
map

In [ ]:
# definindo o roi
roi = map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-43.35, -23.05, -43.19, -22.89)
    map.addLayer(roi)
    map.centerObject(roi)

**Muita cautela aqui**! A produção de um timelapse consome bastante memória, embora ela seja feita *na nuvem*. Isso significa que dependendo da frequência escolhida e das datas, você pode obter uma msg de *User memory limit exceeded*. nestes casos, significa que você te que diminuir a quantidade de imagens que são usadas, modulando o início e o fim, além da frequência.

In [ ]:
timelapse = geemap.sentinel2_timelapse(
    roi,
    out_gif='sentinel2.gif',
    start_year=2024, #<--- se você está acessando esse notebook depois de 2025, ajuste para não exceder o limite da sua memória
#   end_year=2025,  # <--- note que se eu não colocar o ano final, ele vai fazer até o último
    start_date='01-01',
    end_date='12-31',
    frequency='month',
    bands=['SWIR1', 'NIR', 'Red'],
    frames_per_second=3,
    title='Sentinel-2 Timelapse',
)
geemap.show_image(timelapse)

**NOTA**: Se você obtiver um erro aqui, pode ser que a conexão de internet é lenta demais. Lembre-se que esses timelapses estão sendo gerados na nuvem.

Podemos criar uma timeseries genérica, com diferentes tipos de imagens. O exemplo abaixo é *genérico* e pode ser usado com vários diferentes produtos. Note que você **deve** esperar todas as imagens serem carregadas.

In [ ]:
collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filterMetadata(
    'CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10
)

In [ ]:
start_date = '2016-01-01'
end_date = '2022-12-31'
roi = ee.Geometry.BBox(-43.35, -23.05, -43.19, -22.89)

In [ ]:
images = geemap.create_timeseries(
    collection, start_date, end_date, roi, frequency='year', reducer='median'
)
#images

In [ ]:
map = geemap.Map(center=[-22.85, -43.16], zoom=8)

vis_params = {"min": 0, "max": 4000, "bands": ["B8", "B4", "B3"]}
labels = [str(y) for y in range(2016, 2023)]

map.addLayer(images, vis_params, "Sentinel-2", False)
map.add_time_slider(images, vis_params, time_interval=2, labels=labels)
map.centerObject(roi)
map

### Exercício proposto 3 - Timelapse

Crie um timelapse do Sentinel ou Landsat, da área de um reservatório conhecido (por exemplo, a Usina de Belo Monte. Você também pode compartilhar o seu timelapse nas Mídias Sociais usando a hashtag #EarthEngine e #geemap. Veja este exemplo [aqui](https://imgur.com/YaCHvKC).

In [ ]:
# Escreva seu código aqui ou crie um notebook separado